# Manipulation Methods

### 1. `.apply` and `.where`

`.apply` method allows us to apply a function element-wise to every value

In [1]:
import pandas as pd
url = '/Users/rkuma18/Developer/data-science/Pandas/Dataset/vehicles.csv'
df = pd.read_csv(url,low_memory=False)
city_mpg = df.city08
highway_mpg = df.highway08

In [2]:
def get20(val):
    return val > 20

In [3]:
%%timeit
city_mpg.apply(get20)

4.08 ms ± 15.4 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [4]:
%%timeit
city_mpg.gt(20)

22.5 μs ± 103 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [5]:
make = df.make

In [6]:
make

0        Alfa Romeo
1           Ferrari
2             Dodge
3             Dodge
4            Subaru
            ...    
49575        Subaru
49576        Subaru
49577        Subaru
49578        Subaru
49579        Subaru
Name: make, Length: 49580, dtype: object

In [7]:
make.value_counts()

make
Chevrolet                 4569
Ford                      3903
GMC                       2882
Dodge                     2718
BMW                       2643
                          ... 
London Taxi                  1
Excalibur Autos              1
Mahindra                     1
Azure Dynamics               1
Lambda Control Systems       1
Name: count, Length: 146, dtype: int64

The first five entries in the index are the values I want to keep, everythings else I want to replace with Other.

In [8]:
top5 = make.value_counts().index[:5]

def generalize_top5(val):
    if val in top5:
        return val
    return 'Other'

In [9]:
make.apply(generalize_top5)

0        Other
1        Other
2        Dodge
3        Dodge
4        Other
         ...  
49575    Other
49576    Other
49577    Other
49578    Other
49579    Other
Name: make, Length: 49580, dtype: object

-> `.apply` method will call the function for us

-> A faster way of doing this is using the `.where` method. This methods takes a boolean array to make where a condiiton is true

In [10]:
make.where(make.isin(top5), other='Other')

0        Other
1        Other
2        Dodge
3        Dodge
4        Other
         ...  
49575    Other
49576    Other
49577    Other
49578    Other
49579    Other
Name: make, Length: 49580, dtype: object

In [11]:
%%timeit
make.apply(generalize_top5)

24.5 ms ± 132 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [12]:
%%timeit
make.where(make.isin(top5), other='Other')

2.08 ms ± 22.8 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


The complement of the `.where` method is the `.mask` method. 

In [13]:
make.mask(~make.isin(top5), other='Others')

0        Others
1        Others
2         Dodge
3         Dodge
4        Others
          ...  
49575    Others
49576    Others
49577    Others
49578    Others
49579    Others
Name: make, Length: 49580, dtype: object

`~` perform an inversion

### 2. If Else with Pandas

In [15]:
vc = make.value_counts()
top5 = vc.index[:5]
top10 = vc.index[:10]
def generalize(val):
    if val in top5:
        return val
    elif val in top10:
        return 'Top10'
    else:
        return 'Other'

In [16]:
make.apply(generalize)

0        Other
1        Other
2        Dodge
3        Dodge
4        Other
         ...  
49575    Other
49576    Other
49577    Other
49578    Other
49579    Other
Name: make, Length: 49580, dtype: object

In [18]:
(make
    .where(make.isin(top5), 'Top10')
    .where(make.isin(top10), 'Other')
)

0        Other
1        Other
2        Dodge
3        Dodge
4        Other
         ...  
49575    Other
49576    Other
49577    Other
49578    Other
49579    Other
Name: make, Length: 49580, dtype: object

In [19]:
import numpy as np

np.select([make.isin(top5), make.isin(top10)],
            [make, 'Top10'], 'Other')

array(['Other', 'Other', 'Dodge', ..., 'Other', 'Other', 'Other'],
      shape=(49580,), dtype=object)

### 3. Missing Data

In [20]:
cyl = df.cylinders

We convet the property to booleans (using `.isna`), then call `.sum`

In [21]:
(cyl
    .isna()
    .sum()
)

np.int64(1371)

In [22]:
missing = cyl.isna()
make.loc[missing]

7138     Nissan
7139     Toyota
8143     Toyota
8144       Ford
8146       Ford
          ...  
44027     Tesla
44029     Tesla
44030     Tesla
44031    Toyota
44032    Toyota
Name: make, Length: 1371, dtype: object

### 4. Filling In Missing Data

`.fillna` method allows us to specify a replacement value for any missing data.

In [23]:
cyl[cyl.isna()]

7138    NaN
7139    NaN
8143    NaN
8144    NaN
8146    NaN
         ..
44027   NaN
44029   NaN
44030   NaN
44031   NaN
44032   NaN
Name: cylinders, Length: 1371, dtype: float64

In [24]:
cyl.fillna(0).loc[7136:7141]

7136    6.0
7137    6.0
7138    0.0
7139    0.0
7140    6.0
7141    6.0
Name: cylinders, dtype: float64

`data.dropna()`, `data.ffill()`, `data.interpolate()`, `data.ffillna(data.mean())`

### 5. Interpolating Data

This come handy if the data is ordered and there are holes in the data.

In [25]:
temp = pd.Series([32,40,None,42,39,32])

In [26]:
temp

0    32.0
1    40.0
2     NaN
3    42.0
4    39.0
5    32.0
dtype: float64

In [27]:
temp.interpolate()

0    32.0
1    40.0
2    41.0
3    42.0
4    39.0
5    32.0
dtype: float64

### 6. Clipping Data

If we have outliers in our data, we might want to use the `.clip`method

In [28]:
city_mpg.loc[:446]

0      19
1       9
2      23
3      10
4      17
       ..
442    15
443    15
444    15
445    15
446    31
Name: city08, Length: 447, dtype: int64

In [29]:
(city_mpg
    .loc[:446]
    .clip(lower=city_mpg.quantile(.05),
        upper=city_mpg.quantile(.95)))

0      19
1      12
2      23
3      12
4      17
       ..
442    15
443    15
444    15
445    15
446    31
Name: city08, Length: 447, dtype: int64

### 7. Sorting Values

The `.sort_values` method will sort the values in ascending order ans also rearrange the index accordingly

In [30]:
city_mpg.sort_values()

45597      6
34522      6
21057      6
44323      6
7901       6
        ... 
40256    151
42181    151
40625    151
40254    153
40623    153
Name: city08, Length: 49580, dtype: int64

In [31]:
(city_mpg.sort_values() + highway_mpg) /2

0        22.0
1        11.5
2        28.0
3        11.0
4        20.0
         ... 
49575    22.5
49576    24.0
49577    21.0
49578    21.0
49579    18.5
Length: 49580, dtype: float64

### 8. Sorting the Index

To sort the index of a series, we can use the `.sort_index` method

In [32]:
city_mpg.sort_values().sort_index()

0        19
1         9
2        23
3        10
4        17
         ..
49575    19
49576    20
49577    18
49578    18
49579    16
Name: city08, Length: 49580, dtype: int64

### 9. Dropping Duplicates

The `.drop_duplicate` method will remove values that appear more than once

In [33]:
city_mpg.drop_duplicates()

0         19
1          9
2         23
3         10
4         17
        ... 
42195    133
42266     70
42650     56
44030    142
44031    143
Name: city08, Length: 142, dtype: int64

### 10. Ranking

The `.rank` method will return a series that keeps the original index but uses the ranks of values from the original series

In [34]:
city_mpg.rank()

0        30251.5
1          253.0
2        41059.5
3          639.0
4        21588.5
          ...   
49575    30251.5
49576    33475.5
49577    26159.0
49578    26159.0
49579    17061.0
Name: city08, Length: 49580, dtype: float64

In [35]:
city_mpg.rank(method='min')

0        28447.0
1          148.0
2        40140.0
3          359.0
4        19306.0
          ...   
49575    28447.0
49576    32057.0
49577    23872.0
49578    23872.0
49579    14817.0
Name: city08, Length: 49580, dtype: float64

In [37]:
city_mpg.rank(method='dense')

0        14.0
1         4.0
2        18.0
3         5.0
4        12.0
         ... 
49575    14.0
49576    15.0
49577    13.0
49578    13.0
49579    11.0
Name: city08, Length: 49580, dtype: float64

### 11. Replacing

The `.replace` method allows us to map values to new values.

In [38]:
make.replace('Subaru', '昴, すばる')

0        Alfa Romeo
1           Ferrari
2             Dodge
3             Dodge
4            昴, すばる
            ...    
49575        昴, すばる
49576        昴, すばる
49577        昴, すばる
49578        昴, すばる
49579        昴, すばる
Name: make, Length: 49580, dtype: object

### 12. Binning Data

We can bin data as well. Using the `cut` function, we can create bins of equal width.

In [39]:
pd.cut(city_mpg,10)

0        (5.853, 20.7]
1        (5.853, 20.7]
2         (20.7, 35.4]
3        (5.853, 20.7]
4        (5.853, 20.7]
             ...      
49575    (5.853, 20.7]
49576    (5.853, 20.7]
49577    (5.853, 20.7]
49578    (5.853, 20.7]
49579    (5.853, 20.7]
Name: city08, Length: 49580, dtype: category
Categories (10, interval[float64, right]): [(5.853, 20.7] < (20.7, 35.4] < (35.4, 50.1] < (50.1, 64.8] ... (94.2, 108.9] < (108.9, 123.6] < (123.6, 138.3] < (138.3, 153.0]]

In [40]:
pd.cut(city_mpg, [0,10,20,40,70,150])

0        (10, 20]
1         (0, 10]
2        (20, 40]
3         (0, 10]
4        (10, 20]
           ...   
49575    (10, 20]
49576    (10, 20]
49577    (10, 20]
49578    (10, 20]
49579    (10, 20]
Name: city08, Length: 49580, dtype: category
Categories (5, interval[int64, right]): [(0, 10] < (10, 20] < (20, 40] < (40, 70] < (70, 150]]

In [41]:
pd.qcut(city_mpg,10)

0         (18.0, 19.0]
1        (5.999, 13.0]
2         (22.0, 26.0]
3        (5.999, 13.0]
4         (16.0, 17.0]
             ...      
49575     (18.0, 19.0]
49576     (19.0, 20.0]
49577     (17.0, 18.0]
49578     (17.0, 18.0]
49579     (15.0, 16.0]
Name: city08, Length: 49580, dtype: category
Categories (10, interval[float64, right]): [(5.999, 13.0] < (13.0, 15.0] < (15.0, 16.0] < (16.0, 17.0] ... (19.0, 20.0] < (20.0, 22.0] < (22.0, 26.0] < (26.0, 153.0]]